<a href="https://colab.research.google.com/github/godpeny/laboratory/blob/master/laboratory/Study/NLP_Using_Deep_Learning/Word_Embedding/doc_2_vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 44.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 6.7 MB/s eta 0:00:00
  Created wheel for mecab-python: filename=mecab_python-1.0.0-py3-none-any.whl size=1231 sha256=7378c748b2e3fde0014d71804d8c3dad86fff39600ee02b03ea8f6f97357dfb1
  Stored in directory: /root/.cache/pip/wheels/ec/41/c8/ce8fb469be2547ce596c4000613806505dc54c296aae45da71
Successfully built mecab-python
Install mecab-ko
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1381k  100 1381k    0     0   986k      0  0:00:01  0:00:01 --:--:-- 2709k
mecab-0.996-ko-0.9.2/
mecab-0.996-ko-0.9.2/example/
mecab-0.996-ko-0.9.2/example

In [2]:
import pandas as pd

from konlpy.tag import Mecab
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm

from keras.models import Sequential, load_model

from google.colab import drive

# Data Preprocessing

In [3]:
drive.mount('/d')
file_path = '/d/My Drive/Colab Notebooks/NLP_Using_Deep_Learning/data/dart.csv'

df = pd.read_csv(file_path, sep=',')
df = df.dropna()
df.head(10)

Mounted at /d


,code,market,name,business
0,000020,KOSPI,동화약품,II. 사업의 내용\n1. 사업의 개요\n가. 일반적인 사항\n기업회계기준서 제11...
1,000040,KOSPI,KR모터스,II. 사업의 내용\n1. 사업의 개요\n가. 업계의 현황\n수출주력시장인 유럽 불...
2,000050,KOSPI,경방,II. 사업의 내용\n1. 사업의 개요\n(1) 산업의 특성\n[섬유사업부문]\n면...
3,000060,KOSPI,메리츠화재,"II. 사업의 내용\n1. 사업의 개요\n가. 산업의 특성, 성장성, 경기변동의 특..."
4,000070,KOSPI,삼양홀딩스,"II. 사업의 내용\n1. 사업의 개요\n가. 업계의 현황\n지주회사(持株會社, H..."
6,000080,KOSPI,하이트진로,II. 사업의 내용\n1. 사업의 개요\n가. 사업부문별 성과\n당사(연결실체)의 ...
8,000100,KOSPI,유한양행,II. 사업의 내용\n1. 사업의 개요\n가. 업계의 현황\n1) 산업의 특성\n가...
10,000120,KOSPI,CJ대한통운,II. 사업의 내용\n1. 사업의 개요\n가. 사업부문별 종속회사 현황\n사업부문명...
11,000140,KOSPI,하이트진로홀딩스,II. 사업의 내용\n1. 사업의 개요\n당사는 한국채택국제회계기준(K-IFRS) ...
13,000150,KOSPI,두산,II. 사업의 내용\n1. 사업의 개요\n가. 사업부문별 주요 재무현황\n(1) 사...


In [15]:
mecab = Mecab()

tagged_corpus_list = []

for idx, row in df.iterrows():
    tagged_corpus_list.append(TaggedDocument(tags=[row['name']], words=mecab.morphs(row['business'])))

print(len(tagged_corpus_list))

2295


In [16]:
# check tagged_corpus
print(tagged_corpus_list[0])

TaggedDocument<['II', '.', '사업', '의', '내용', '1', '.', '사업', '의', '개요', '가', '.', '일반', '적', '인', '사항', '기업', '회계', '기준', '서', '제', '1110', '호', '"', '연결', '재무제표', '"', '의', '의하', '여', '2018', '년', '12', '월', '17', '일', '에', '설립', '한', '동화', '크립톤', '기업가', '정신', '제일', '호', '창업', '벤처', '전문', '사모', '투자', '합자회사', '를', '종속', '회사', '에', '편입', '하', '였', '습니다', '.', '나', '.', '지배', '기업', '의', '현황', '1', ')', '산업', '의', '특성', '제약', '산업', '은', '약학', ',', '화학', ',', '생물학', '등', '여러', '분야', '의', '지식', '과', '기술', '이', '복합', '적', '으로', '필요', '한', '기술', '·', '지식', '집약', '형', '산업', '으로', '인간', '의', '생명', '과', '보건', '에', '직접', '적', '으로', '관련', '된', '제품', '을', '생산', '하', '고', ',', '개발', '·', '제조', '·', '유통', '·', '가격', '등', '을', '정부', '에서', '엄격', '하', '게', '규제', '하', '고', '관리', '하', '는', '산업', '적', '특성', '을', '가지', '고', '있', '습니다', '.', '또한', '신약', '개발', '에', '는', '막대', '한', '시간', '과', '비용', '이', '소요', '되', '며', ',', '성공', '확률', '또한', '높', '지', '않', '으나', '신약', '개발', '에', '성공', '할', '경우', '엔', '엄청난', '부'

# Modeling

In [17]:
from gensim.models import doc2vec

In [18]:
embedding_dims = 300

model = doc2vec.Doc2Vec(vector_size=embedding_dims, alpha=0.025, min_alpha=0.025, workers=8, window=8)
model.build_vocab(tagged_corpus_list)

### Doc2Vec
 - vector_size : Dimensionality of the feature vectors.
 - alpha : The initial learning rate.
 - min_alpha :  Learning rate will linearly drop to min_alpha as training progresses.
 - window : The maximum distance between the current and predicted word within a sentence.
 - workers : Use these many worker threads to train the model (=faster training with multicore machines).



In [19]:
model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=50)

In [14]:
# save model
model_path = '/d/My Drive/Colab Notebooks/NLP_Using_Deep_Learning/models/dart'
model.save(model_path)

In [20]:
similar_doc = model.docvecs.most_similar('동화약품')
print(similar_doc)

[('KR모터스', 0.9999721050262451), ('CJ대한통운', 0.9999608397483826), ('삼양홀딩스', 0.9999410510063171), ('하이트진로', 0.9998406767845154), ('메리츠화재', 0.9997520446777344), ('유한양행', 0.9993683099746704), ('경방', 0.9993384480476379), ('대림산업', 0.9988945722579956), ('기아차', 0.998472273349762), ('중앙에너비스', 0.9982314705848694)]


<ipython-input-20-8152187b51f2>:1: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  similar_doc = model.docvecs.most_similar('동화약품')
